In [123]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from poisson_binomial import poisson_binomial_pmf
from scipy import stats

# Prior on mu - log prior and gradient

In [216]:
bins = np.array([0.0, 5.0, 16.0, 17.0])
G = 0.5*(bins[1:]+bins[:-1])
G2 = np.square(G[:,np.newaxis]-G[np.newaxis,:])
M = bins.size-1

mu = np.array([-1.2,3.4,0.3])
m = 1.2
tau2 = 0.3**2.0


def log_prior(mu,m,tau2):
    J = tau2*np.exp(-G2/2.0/m**2)
    J_inv = np.linalg.pinv(J)
    J_inv_mu = np.dot(J_inv,mu)
    
    lnP = +0.5*np.linalg.slogdet(J_inv)[1]-0.5*np.dot(mu.T,J_inv_mu) - (M/2)*np.log(2.0*np.pi)
    
    dJdm = J*G2/m**3
    dJdtau2 = J/tau2

    dlnPdm = -0.5*np.trace(np.dot(J_inv,dJdm))+0.5*np.dot(J_inv_mu.T,np.dot(dJdm,J_inv_mu))
    dlnPdtau2 = -0.5*np.trace(np.dot(J_inv,dJdtau2))+0.5*np.dot(J_inv_mu.T,np.dot(dJdtau2,J_inv_mu))
    dlnPdmu = -J_inv_mu
    
    return lnP, dlnPdmu, dlnPdm, dlnPdtau2

def check(x,y,z):
    print(x,y,(x-y)/z)

lnP, dlnPdmu, dlnPdm, dlnPdtau2 = log_prior(mu,m,tau2)
lnP_truth = stats.multivariate_normal(mean=np.zeros(M),cov=J,allow_singular=True).logpdf(mu)
check(lnP,lnP_truth,lnP_truth)
h = 1e-7

lnP_dmu = log_prior(mu+h*np.array([1,0,0]),m,tau2)[0]
check(lnP+h*dlnPdmu[0],lnP_dmu,lnP_truth)

lnP_dm = log_prior(mu,m+h,tau2)[0]
check(lnP+h*dlnPdm,lnP_dm,lnP_truth)

lnP_dtau2 = log_prior(mu,m,tau2+h)[0]
check(lnP+h*dlnPdtau2,lnP_dtau2,lnP_truth)

-71.86707718447357 -71.86707718447353 5.932141088216752e-16
-71.86707585114024 -71.86707585114029 -5.932141088216752e-16
-71.86707718438562 -71.8670771843856 3.9547607254778346e-16
-71.86699804871802 -71.8669980488068 -1.2354672506392755e-12


# Global

In [218]:
#bins = np.array([0.0, 5.0, 16.0, 17.0, 18.0, 18.2, 18.4, 18.6, 18.8, 19.0, 19.1, 19.2, 19.3, 19.4, 19.5, 19.6, 19.7, 19.8, 19.9, 20.0, 20.1, 20.2, 20.3, 20.4, 20.5, 20.6, 20.7, 20.8, 20.9, 21.0, 21.1, 21.2, 21.3, 21.4, 21.5, 25.0])
bins = np.array([0.0, 5.0, 16.0, 17.0])
midbins = 0.5*(bins[1:]+bins[:-1])
D2 = np.square(midbins[:,np.newaxis]-midbins[np.newaxis,:])
N_bins = bins.size-1
N_times = 1234

tmp_args = np.concatenate([np.array([4.0,0.5,4.0,0.5]),np.ones(N_bins),np.random.normal(0,1,N_bins*N_times)])

def global_log_likelihood(args):
    
    # Unpack parameters
    log_l, log_sigma2, log_m, log_tau2 = args[:4]
    mu = args[4:4+N_bins]
    x = args[4+N_bins:]
    
    l2 = np.exp(-2.0*log_l)
    sigma2 = np.exp(log_sigma2)
    p = 1.0/(1.0+np.exp(-x))
    
    ##### Priors
    log_likelihood = np.zeros(args.size)
    
    # mu prior
    Sigma = sigm2*np.exp(-D2/l2/2.0)
    Sigma_inverse = np.linalg.pinv(Sigma)
    log_likelihood np.dot(mu.T,)
    
    # Pass mu, x and p out to the child processes
    

SyntaxError: invalid syntax (<ipython-input-218-a1c14bf2d7c3>, line 27)

In [3]:
def poisson_binomial_pmf_easy(probs):
    probslen = probs.size
    result = np.zeros(probslen+1)
    poisson_binomial_pmf(probs,probslen,result)
    return result

p = np.array([0.3,0.6,0.2,0.1,0.9,0.5])
n = p.size
pmf = np.zeros(n+1)
poisson_binomial_pmf(p,n,pmf)
print(pmf,pmf.sum())
print(poisson_binomial_pmf_easy(p))

[0.01008 0.12388 0.3353  0.3484  0.153   0.02772 0.00162] 1.0
[0.01008 0.12388 0.3353  0.3484  0.153   0.02772 0.00162]


In [4]:
from numba import njit

@njit
def poisson_binomial_likelihood(k,probs,probslen,pmf,subpmf,likelihood,gradient):
    
    # Compute the pmf
    poisson_binomial_pmf(probs,probslen,pmf)
    likelihood[0] = pmf[k]
    
    for i in range(probslen):
        p = probs[i]
        subpmf[0] = pmf[0]/(1.0-p)
        for j in range(probslen-1):
            subpmf[j] = (pmf[j]-subpmf[j-1]*p)/(1.0-p)
        subpmf[probslen-1] = pmf[probslen-1]/p
        gradient[i] = subpmf[k-1]-subpmf[k]
    

In [7]:
from numba import njit
            
@njit
def poisson_binomial_likelihood(k,probs,probslen,pmf,subpmf,likelihood,gradient):
    
    # Compute the pmf
    poisson_binomial_pmf(probs,probslen,pmf)
    likelihood[0] = pmf[k]
    
    
    if k == 0:
        gradient_first_term,gradient_second_term=0.0,1.0
    elif k == probslen:
        gradient_first_term,gradient_second_term=1.0,0.0
    else:
        gradient_first_term,gradient_second_term=1.0,1.0
    
    for i in range(probslen):
        
        p = probs[i]
        oneoveroneminusp = 1.0/(1.0-p)
        
        subpmf[0] = pmf[0]*oneoveroneminusp
        for j in range(1,probslen):
            subpmf[j] = (pmf[j]-subpmf[j-1]*p)*oneoveroneminusp
        subpmf[probslen-1] = pmf[probslen]/p
        
        gradient[i] = gradient_first_term*subpmf[k-1]-gradient_second_term*subpmf[k]
        
    

k = 5
subpmf = np.zeros(n)
gradient = np.zeros(n)
likelihood = np.zeros(1)
poisson_binomial_likelihood(k,p,n,pmf,subpmf,likelihood,gradient)

dp = 1e-4
print(likelihood[0])
print(poisson_binomial_pmf_easy(p+dp*np.array([0,0,0,1,0,0]))[k])
print(likelihood[0]+dp*gradient[3])

0.027719999999999998
0.02773152
0.02773152


In [11]:
dp = 1e-7*np.random.uniform(-1,1,6)
print(likelihood[0])
print(poisson_binomial_pmf_easy(p+dp)[k])
print(likelihood[0]+np.dot(dp,gradient))

0.027719999999999998
0.027720027559628026
0.0277200275596198


In [17]:
from numba import njit
            
@njit
def poisson_binomial_log_likelihood(k,probs,probslen,pmf,subpmf,log_likelihood,gradient):
    
    # Compute the pmf
    poisson_binomial_pmf(probs,probslen,pmf)
    likelihood = pmf[k]
    log_likelihood[0] = np.log(likelihood)
    
    
    if k == 0:
        gradient_first_term,gradient_second_term=0.0,1.0
    elif k == probslen:
        gradient_first_term,gradient_second_term=1.0,0.0
    else:
        gradient_first_term,gradient_second_term=1.0,1.0
    
    for i in range(probslen):
        
        p = probs[i]
        oneoveroneminusp = 1.0/(1.0-p)
        
        subpmf[0] = pmf[0]*oneoveroneminusp
        for j in range(1,probslen):
            subpmf[j] = (pmf[j]-subpmf[j-1]*p)*oneoveroneminusp
        subpmf[probslen-1] = pmf[probslen]/p
        
        gradient[i] = (gradient_first_term*subpmf[k-1]-gradient_second_term*subpmf[k])/likelihood
        
    

k = 4
subpmf = np.zeros(n)
gradient = np.zeros(n)
log_likelihood = np.zeros(1)
poisson_binomial_log_likelihood(k,p,n,pmf,subpmf,log_likelihood,gradient)

dp = 1e-7*np.random.uniform(-1,1,6)
print(log_likelihood[0])
print(np.log(poisson_binomial_pmf_easy(p+dp)[k]))
print(log_likelihood[0]+np.dot(dp,gradient))

-1.8773173575897015
-1.8773177694757819
-1.877317769475694


In [113]:
N = 5
x = np.arange(N)
l = 1.0
tau = np.abs(x[:,np.newaxis]-x[np.newaxis,:])
K = np.exp(-tau/l)
K_inv = np.linalg.pinv(K)
K_inv[np.abs(K_inv) < 1e-14] = 0.0
print(K_inv)
print(np.linalg.det(K_inv))
u = np.exp(1.0/l)
a = u*u
b = u*u+1
c = -u
#print(a,b,c)
J_inv = np.zeros(K.shape)
J_inv[0,0] = J_inv[-1,-1] = a
for i in range(1,N-1):
    J_inv[i,i] = b
for i in range(0,N-1):
    J_inv[i,i+1] = J_inv[i+1,i] = c
J_inv /= u*u-1
print(J_inv)
print(np.power(u*u/(u*u-1),N-1))

[[ 1.15651764 -0.42545906  0.          0.          0.        ]
 [-0.42545906  1.31303529 -0.42545906  0.          0.        ]
 [ 0.         -0.42545906  1.31303529 -0.42545906  0.        ]
 [ 0.          0.         -0.42545906  1.31303529 -0.42545906]
 [ 0.          0.          0.         -0.42545906  1.15651764]]
1.7889946812194075
[[ 1.15651764 -0.42545906  0.          0.          0.        ]
 [-0.42545906  1.31303529 -0.42545906  0.          0.        ]
 [ 0.         -0.42545906  1.31303529 -0.42545906  0.        ]
 [ 0.          0.         -0.42545906  1.31303529 -0.42545906]
 [ 0.          0.          0.         -0.42545906  1.15651764]]
1.788994681219407


In [72]:
for i in range(N):
    print(i+1,np.linalg.det(K_inv[:i+1,:i+1]))

1 1.1565176427496664
2 1.3375330579912446
3 1.546880579327785
4 1.7889946812194075
5 2.0690039116155585
6 2.0690039116155563


In [77]:
b

1.1565176427496655

In [101]:
def true_det(i):
    return np.linalg.det(K_inv[:i,:i])

u = np.exp(1.0/l)
i = 6
print(np.power(u*u/(u*u-1),i))
print(true_det(i))
print(c*true_det(i-1)-a*a*true_det(i-2))

2.3928395267014633
2.0690039116155563
2.392839526701463


In [107]:
1-1.0/(u*u)

0.8646647167633873

In [95]:
K_inv[:1+1,:1+1]

array([[ 1.15651764, -0.42545906],
       [-0.42545906,  1.31303529]])

In [68]:
np.linalg.det(K_inv[:5,:5])

1.7889946812194075

In [42]:
-np.exp(1.0/l)/(np.exp(2.0/l)-1)

-0.959517375667472

In [46]:
np.exp(2.0/l)/(np.exp(2.0/l)-1)

1.1565176427496657